# Лабортаторная работа 4. Графы

## Задание 4. Реализовать в виде программ алгоритмы, рассмотренные на лекции:а)задача Прима-Крускала;б)найти число различных путей из одной данной вершины в другую; решить задачу, используя списки смежности.Граф задан списком ребер, хранящемся в файле.

In [10]:
import typing as tp
import pprint

with open(r'input4.txt') as file:
	graph4 = file.read().splitlines()
print(f'Исходный граф: {graph4}')


def get_adjacency_matrix_(input_list: tp.List[str]) -> tp.List[list]:
	""" Функция, формирующая матрицу весов """
	splitted_list = list(map(lambda x: x.split(), input_list))
	num_nested_list = [list(map(int, num)) for num in splitted_list]
	filtered_list = list(filter(lambda x: len(x) > 1, num_nested_list))
	zeros_nested_list = [[0] * num_nested_list[0][0] for _ in range(num_nested_list[0][0])]
	for edge in filtered_list:
		row = edge[0]
		col = edge[1]
		zeros_nested_list[row - 1][col - 1] = 1
	return zeros_nested_list


adjacency_matrix = get_adjacency_matrix_(graph4)
pprint.pprint(adjacency_matrix)


def kruskal_algorithm(input_list: tp.List[str]) -> tp.List[list]:
	""" Функция, реализующая алгоритм Краскала """
	splitted_list = list(map(lambda x: x.split(), input_list))
	filtered_list = list(filter(lambda x: len(x) > 1, splitted_list))
	sorted_list = sorted([list(map(int, num)) for num in filtered_list], key=lambda x: x[2])
	connected_nodes_list = set()   															# список соединенных вершин
	isolated_nodes_dict = {}      															# словарь изолированных групп вершин
	edges_list = []																			# список ребер остова
	for r in sorted_list:
		if r[0] not in connected_nodes_list or r[1] not in connected_nodes_list:  			# проверка для исключения циклов в остове
			if r[0] not in connected_nodes_list and r[1] not in connected_nodes_list:	 	# если обе вершины не соединены, то
				isolated_nodes_dict[r[0]] = [r[0], r[1]]          							# формируем в словаре ключ с номерами вершин
				isolated_nodes_dict[r[1]] = isolated_nodes_dict[r[0]]               		# и связываем их с одним и тем же списком вершин
			else:                           												# иначе
				if not isolated_nodes_dict.get(r[0]):             							# если в словаре нет первой вершины, то
					isolated_nodes_dict[r[1]].append(r[0])        							# добавляем в список первую вершину
					isolated_nodes_dict[r[0]] = isolated_nodes_dict[r[1]]           		# и добавляем ключ с номером первой вершины
				else:
					isolated_nodes_dict[r[0]].append(r[1])        							# иначе, все то же самое делаем со второй вершиной
					isolated_nodes_dict[r[1]] = isolated_nodes_dict[r[0]]
			edges_list.append(r)             												# добавляем ребро в остов
			connected_nodes_list.add(r[0])             										# добавляем вершины в множество
			connected_nodes_list.add(r[1])
	for r in sorted_list:    																# проходим по ребрам второй раз и объединяем разрозненные группы вершин
		if r[1] not in isolated_nodes_dict[r[0]]:     										# если вершины принадлежат разным группам, то объединяем
			edges_list.append(r)             												# добавляем ребро в остов
			gr1 = isolated_nodes_dict[r[0]]
			isolated_nodes_dict[r[0]] += isolated_nodes_dict[r[1]]      					# объединяем списки двух групп вершин
			isolated_nodes_dict[r[1]] += gr1
	return edges_list


print(kruskal_algorithm(graph4))


def get_path_count(input_list: tp.List[str], start_node, end_node, visited):
	""" Количество путей """
	if start_node == end_node:
		return 1
	visited.append(start_node)
	count = 0
	for v in input_list[start_node]:
		if not v in visited:
			count += get_path_count(input_list, v, end_node, visited)
	visited.pop()
	return count


print(f'Количество путей из данной вершины: {get_path_count(adjacency_matrix, 1, 3, [])}')

Исходный граф: ['6', '9', '1 2 2', '1 4 7', '2 3 4', '3 1 2', '3 4 13', '4 5 1', '5 3 12', '5 6 5', '6 3 2']
[[0, 1, 0, 1, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [1, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 1],
 [0, 0, 1, 0, 0, 0]]
[[4, 5, 1], [1, 2, 2], [3, 1, 2], [6, 3, 2], [5, 6, 5]]
Количество путей из данной вершины: 0


## Задание 5 (выполняется по вариантам). Составить  программы, решающие следующие задачи. Входные данные находятся в файле input.txt. Для каждой задачи для отчета по работе изобразить на бумаге/в файле отчета граф, соответствующий данным в файле input.txt.

### 1. Туристическое агентство организует путешествия по городам мира. Прямые авиарейсы существуют только между некоторыми из них, причем необязательно в обоих направлениях. Требуется найти все возможные маршруты, начинающиеся в городе А и заканчивающиеся в городе В, что туристу, купившему путевку, не придется посещать один и тот же город дважды,а также подсчитать количество таких маршрутов. Технические требования:Количество городов N <= 10 Формат файла INPUT.TXT:1-я строка –количество городов; 2-я –названия городов А и В через  пробел; последующие строки содержат названия пар городов X и Y через пробел, для которых имеется авиарейс из X в Y.

In [45]:
with open(r'input5.txt') as file5:
	graph5 = file5.read().splitlines()
print(f'Исходные данные: {graph5}')

d = {}													# создаем пустой словарь
for name in graph5[1].split():							# разбиваем 2 строку на подстроки так как там все города по условию
	d[name] = []										# создаем по ключам пустые списки
for i in range(2, len(graph5)):
	a, b = graph5[i].split()							# разбиваем 2 города
	d[a].append(b)										# добавляем 1 город в ключ другой в значение
print(d)

def get_possible_routes(start, end, visited = []):		# алгоритм похож на модифицированный поиск в глубину
	n_visit = visited.copy()
	n_visit.append(start)
	if start == end:
		print(*n_visit)
		return
	for city in d[start]:
		if city not in n_visit:
			get_possible_routes(city, end, n_visit)
print(get_possible_routes('a', 'c'))

Исходные данные: ['4', 'a b c d', 'a b', 'b c', 'c d', 'd a']
{'a': ['b'], 'b': ['c'], 'c': ['d'], 'd': ['a']}
a b c
None
